In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms

import torch.optim as optim

from tqdm import  tqdm
import os

In [2]:
LEARNING_RATE = 1e-4
BATCH_SIZE = 512
MAX_EPOCHS = 200

EVAL_FREQ = 500

# ADAM RMSprop SGD
OPTIMIZER = 'ADAM'

In [3]:
transform_train = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])
 
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4941, 0.4850, 0.4502), (0.2467, 0.2429, 0.2616))])
    
    
trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform_train)

trainloader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=2)
 
testset = torchvision.datasets.CIFAR10(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform_test)

testloader = torch.utils.data.DataLoader(
    testset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=2)
 
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

/home/ma-user/anaconda3/envs/PyTorch-1.8/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class CNN(nn.Module):
    def __init__(self, n_channels, n_classes): 
        super().__init__()
        self.features = nn.Sequential(
            #1
            nn.Conv2d(in_channels=n_channels, out_channels=64, kernel_size=(3,3), stride=1, padding=1, bias=True),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            #2
            nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1),
            #3
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=1, padding=1, bias=True),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            #4
            nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1),
            #5
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=1, padding=1, bias=True),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            #6
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=1, padding=1, bias=True),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            #7
            nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1),
            #8
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), stride=1, padding=1, bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            #9
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=1, padding=1, bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            #10
            nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1),
            #11
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=1, padding=1, bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            #12
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=1, padding=1, bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            #13
            nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1),
            
        )
        self.classifier = nn.Sequential(
            nn.Linear(512, n_classes)
        )
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
 
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x 
        

In [11]:
def train(optimizer, learning_rate, max_epochs, eval_freq, ada_opt, save_para):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
    print("start training CNN...")
    print(device)
    # 储存学习到的参数
    opt_save_path = "optimizer" + optimizer + '.pth'
    writer = SummaryWriter('./logs-CNN-batch512/' + optimizer + '/lr=' + str(learning_rate))
    
    model = CNN(n_channels=3, n_classes=10)
    model.to(device)
    
    cost = nn.CrossEntropyLoss()
    
    if save_para and os.path.exists(opt_save_path):
      model.load_state_dict(torch.load(opt_save_path))
    
    if optimizer == 'SGD':
      optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    elif optimizer == 'ADAM':
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer == 'RMSprop':
      optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    
    if ada_opt:
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80, 100, 120], gamma=0.1)
    
    for epoch in tqdm(range(max_epochs)):
        
        model.train()
        
        train_acc_avg = 0.
        train_loss_avg = 0.
        
        for idx, (image, label) in enumerate(trainloader):
            
            image, label = image.to(device), label.to(device)
            
            optimizer.zero_grad()
            
            output = model(image)
            loss = cost(output, label)
            loss.backward()
            optimizer.step()
            
            train_loss_avg = train_loss_avg + loss.item()
            
            _, pred = output.max(1)
            num_correct = (pred == label).sum().item()
            train_acc_avg = train_acc_avg + num_correct/label.shape[0]
            
        train_loss_avg = train_loss_avg/len(trainloader)
        train_acc_avg = train_acc_avg/len(trainloader)
        print("epoch is %d, train_loss is %lf, train_acc is %lf" %(epoch+1, train_loss_avg, train_acc_avg))
        writer.add_scalar('Train/Accuracy', train_acc_avg, epoch)
        writer.add_scalar('Train/Loss', train_loss_avg, epoch)
        if ada_opt: scheduler.step()
        
        test_acc_avg = 0.
        test_loss_avg = 0.
        
        with torch.no_grad():
            for idx, (image, label) in enumerate(testloader):
                
                image, label = image.to(device), label.to(device)
                
                output = model(image)
                loss = cost(output, label)
                
                test_loss_avg = test_loss_avg + loss.item()
                
                _, pred = output.max(1)
                num_correct = (pred == label).sum().item()
                test_acc_avg = test_acc_avg + num_correct/label.shape[0]
            
        test_loss_avg = test_loss_avg/len(testloader)
        test_acc_avg = test_acc_avg/len(testloader)
        print('epoch is %d, test_loss is %lf, test_acc is %lf' %(epoch+1, test_loss_avg, test_acc_avg))
        writer.add_scalar('Test/Accuracy', test_acc_avg, epoch)
        writer.add_scalar('Test/Loss', test_loss_avg, epoch)
        
    if save_para: torch.save(model.state_dict(), opt_save_path)
    
    return model
    

In [6]:
#default para defalut opt
model = train(optimizer='ADAM', learning_rate=1e-4, max_epochs=200, eval_freq=500, ada_opt=True, save_para=True)

start training CNN...
cuda:0


  0%|          | 0/200 [00:00<?, ?it/s]

epoch is 1, train_loss is 1.321407, train_acc is 0.530495


  0%|          | 1/200 [00:07<26:07,  7.88s/it]

epoch is 1, test_loss is 1.077020, test_acc is 0.613793
epoch is 2, train_loss is 0.885150, train_acc is 0.688343


...

epoch is 198, test_loss is 1.747480, test_acc is 0.728550
epoch is 199, train_loss is 0.000007, train_acc is 1.000000


100%|█████████▉| 199/200 [19:47<00:05,  5.70s/it]

epoch is 199, test_loss is 1.737781, test_acc is 0.730457
epoch is 200, train_loss is 0.000008, train_acc is 1.000000


100%|██████████| 200/200 [19:53<00:00,  5.97s/it]

epoch is 200, test_loss is 1.757130, test_acc is 0.727987


In [7]:
#default para SGD opt
model = train(optimizer='SGD', learning_rate=1e-4, max_epochs=200, eval_freq=500, ada_opt=True, save_para=True)

start training CNN...
cuda:0


  0%|          | 0/200 [00:00<?, ?it/s]

epoch is 1, train_loss is 2.726918, train_acc is 0.136061


  0%|          | 1/200 [00:05<18:52,  5.69s/it]

epoch is 1, test_loss is 2.442461, test_acc is 0.165338
epoch is 2, train_loss is 2.291968, train_acc is 0.200382


...

epoch is 198, test_loss is 1.290539, test_acc is 0.543997
epoch is 199, train_loss is 1.093301, train_acc is 0.628162


100%|█████████▉| 199/200 [18:59<00:05,  5.82s/it]

epoch is 199, test_loss is 1.289963, test_acc is 0.541446
epoch is 200, train_loss is 1.092865, train_acc is 0.628258


100%|██████████| 200/200 [19:05<00:00,  5.73s/it]

epoch is 200, test_loss is 1.286940, test_acc is 0.544129


In [8]:
#default para RMS opt
model = train(optimizer='RMSprop', learning_rate=1e-4, max_epochs=200, eval_freq=500, ada_opt=True, save_para=True)

start training CNN...
cuda:0


  0%|          | 0/200 [00:00<?, ?it/s]

epoch is 1, train_loss is 1.400436, train_acc is 0.513301


  0%|          | 1/200 [00:05<19:30,  5.88s/it]

epoch is 1, test_loss is 0.999704, test_acc is 0.642837
epoch is 2, train_loss is 0.851914, train_acc is 0.699147


...

epoch is 198, test_loss is 2.034463, test_acc is 0.776132
epoch is 199, train_loss is 0.000000, train_acc is 1.000000


100%|█████████▉| 199/200 [21:41<00:07,  7.57s/it]

epoch is 199, test_loss is 2.023935, test_acc is 0.777171
epoch is 200, train_loss is 0.000000, train_acc is 1.000000


100%|██████████| 200/200 [21:49<00:00,  6.55s/it]

epoch is 200, test_loss is 2.019516, test_acc is 0.776919


In [9]:
#lr=1e-3 defalut opt
model = train(optimizer='ADAM', learning_rate=1e-3, max_epochs=200, eval_freq=500, ada_opt=True, save_para=True)

start training CNN...
cuda:0


  0%|          | 0/200 [00:00<?, ?it/s]

epoch is 1, train_loss is 1.322207, train_acc is 0.563699


  0%|          | 1/200 [00:07<24:55,  7.52s/it]

epoch is 1, test_loss is 0.864261, test_acc is 0.696875
epoch is 2, train_loss is 0.741495, train_acc is 0.739552


...

epoch is 198, test_loss is 1.042501, test_acc is 0.857250
epoch is 199, train_loss is 0.000020, train_acc is 1.000000


100%|█████████▉| 199/200 [20:50<00:05,  5.71s/it]

epoch is 199, test_loss is 1.050467, test_acc is 0.856129
epoch is 200, train_loss is 0.000019, train_acc is 1.000000


100%|██████████| 200/200 [20:56<00:00,  6.28s/it]

epoch is 200, test_loss is 1.049155, test_acc is 0.857646


In [10]:
#lr=1e-5 para defalut opt
model = train(optimizer='ADAM', learning_rate=1e-5, max_epochs=200, eval_freq=500, ada_opt=True, save_para=True)

start training CNN...
cuda:0


  0%|          | 0/200 [00:00<?, ?it/s]

epoch is 1, train_loss is 0.000020, train_acc is 1.000000


  0%|          | 1/200 [00:05<19:06,  5.76s/it]

epoch is 1, test_loss is 1.061776, test_acc is 0.858002
epoch is 2, train_loss is 0.000014, train_acc is 1.000000


...

epoch is 198, test_loss is 1.285650, test_acc is 0.862155
epoch is 199, train_loss is 0.000000, train_acc is 1.000000


100%|█████████▉| 199/200 [20:51<00:05,  5.70s/it]

epoch is 199, test_loss is 1.288942, test_acc is 0.861259
epoch is 200, train_loss is 0.000000, train_acc is 1.000000


100%|██████████| 200/200 [20:57<00:00,  6.29s/it]

epoch is 200, test_loss is 1.281958, test_acc is 0.858812


In [12]:
#lr=1e-5 para defalut opt
model = train(optimizer='ADAM', learning_rate=1e-3, max_epochs=200, eval_freq=500, ada_opt=True, save_para=False)

start training CNN...
cuda:0


  0%|          | 0/200 [00:00<?, ?it/s]

epoch is 1, train_loss is 1.388532, train_acc is 0.499202


  0%|          | 1/200 [00:05<19:27,  5.86s/it]

epoch is 1, test_loss is 0.984427, test_acc is 0.645060
epoch is 2, train_loss is 0.836617, train_acc is 0.703318


...

epoch is 198, test_loss is 1.054656, test_acc is 0.856635
epoch is 199, train_loss is 0.000022, train_acc is 1.000000


100%|█████████▉| 199/200 [21:01<00:05,  5.85s/it]

epoch is 199, test_loss is 1.049911, test_acc is 0.855618
epoch is 200, train_loss is 0.000024, train_acc is 1.000000


100%|██████████| 200/200 [21:06<00:00,  6.33s/it]

epoch is 200, test_loss is 1.051665, test_acc is 0.855819


In [13]:
#lr=1e-5 para defalut opt
model = train(optimizer='ADAM', learning_rate=1e-5, max_epochs=200, eval_freq=500, ada_opt=True, save_para=False)

start training CNN...
cuda:0


  0%|          | 0/200 [00:00<?, ?it/s]

epoch is 1, train_loss is 1.950363, train_acc is 0.346748


  0%|          | 1/200 [00:05<19:09,  5.77s/it]

epoch is 1, test_loss is 1.534186, test_acc is 0.451844
epoch is 2, train_loss is 1.375649, train_acc is 0.512724


...

epoch is 198, test_loss is 1.664768, test_acc is 0.659409
epoch is 199, train_loss is 0.000841, train_acc is 1.000000


100%|█████████▉| 199/200 [21:21<00:05,  5.72s/it]

epoch is 199, test_loss is 1.655259, test_acc is 0.657772
epoch is 200, train_loss is 0.000822, train_acc is 1.000000


100%|██████████| 200/200 [21:26<00:00,  6.43s/it]

epoch is 200, test_loss is 1.654962, test_acc is 0.659754
